# Attempt to tabulate genetable

In [55]:
import requests
import numpy as np
import pandas as pd

In [56]:
disc_genes="T:\colin\BRice\V2\gwas-gene-discovery\MLM.DTF.GWAS.Results\summary_genes_discovered.txt"
list="T:\colin\BRice\V2\gwas-gene-discovery\mock_keyword_list.txt"

In [67]:
def knetapi(disc_genes):
    '''Use the genes from the summary file to search for Knetminer genome api.'''
    with open(disc_genes, "r") as fs:
        next(fs)
        genes = []
        with open(list, "r") as fl:
            pheno = []
            for line in fl:
                pheno.append(line.rstrip())
            #print("The phenotype of interests are {}".format(pheno))
            for line in fs:
                col = line.split("\t")
                genes.append(col[0])
            genelist = (",").join(genes) #join all iterative elements by ,
            #print(genelist)

            #use str.join() to convert multiple elments in a list into one string.
            keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
            link = "http://knetminer.rothamsted.ac.uk/riceknet/genome?"
            parameters = {"keyword":keyw1, "list":genelist}
            r = requests.get(link, params=parameters)

            #check if request is succesfsul.
            if not r.ok:
                r.raise_for_status()
                sys.exit()

            #printing out genetable to a file.
            decoded=r.json()[u'geneTable'].split("\t")
            #remove space or newline at the end.
            decoded=(decoded)[:-1]
            
            ncolumns=9
            #tabulate the list to be any dimension as long as it's dividable by 9 columns.
            genetable=np.array(decoded).reshape(len(decoded)/ncolumns, ncolumns)
            genetable = pd.DataFrame(genetable[1:,:], columns=genetable[0,:])
                
            genes=genetable[u'ACCESSION']
            knetscore=genetable[u'SCORE']
            genechr=genetable[u'CHRO']
            start=genetable[u'START']
            
            return genetable
        
            
            #print(genes)
            #print(knetscore)
            #print(start)            

In [66]:
knetapi(disc_genes)

ONDEX-ID     ACCESSION  \
0                                    EVIDENCE\n47205  OS03G0122600   
1  Phenotype__1__Flowering time independent of......  OS06G0156700   
2               Publication__1__PMID:16133216\n74416  OS06G0156600   
3               Publication__1__PMID:16133216\n74605  OS06G0164900   
4                Publication__1__PMID:1681543\n74630  OS06G0166100   

      GENE NAME CHRO    START  TAXID   SCORE USER  \
0        MADS50    3  1270320  39947  105.49  yes   
1  OS06G0156700    6  2898787  39947    3.69  yes   
2  OS06G0156600    6  2893694  39947    3.69  yes   
3  OS06G0164900    6  3286611  39947    3.67  yes   
4  OS06G0166100    6  3322909  39947    0.00  yes   

                                                 QTL  
0  qtaro_301//qtaro_301||qtaro_531//qtaro_531||qt...  
1  qtaro_447//qtaro_447||qtaro_4//qtaro_4||qtaro_...  
2  qtaro_447//qtaro_447||qtaro_4//qtaro_4||qtaro_...  
3  qtaro_399//qtaro_399||qtaro_447//qtaro_447||qt...  
4  qtaro_399//qtaro_399||qtaro_447//qtaro_447||qt...

### block prototype

In [58]:
import requests
import numpy as np
import pandas as pd

In [3]:
disc_genes="T:\colin\BRice\V2\gwas-gene-discovery\MLM.DTF.GWAS.Results\summary_genes_discovered.txt"
lists="T:\colin\BRice\V2\gwas-gene-discovery\mock_keyword_list.txt"
filtered_snps="T:\colin\BRice\V2\gwas-gene-discovery\MLM.DTF.GWAS.Results\filtered_snps.txt"

In [7]:
def summary(filtered_snps, disc_genes):
    '''Query on ensembl for genes ocurring within a specified distance in bp of each significant SNPs.
    Produce a summary file of the genes as well as the phenotypes and functions linked to the genes provided by ensembl.'''
    with open(filtered_snps, "r") as fr:
        next(fr)
        with open(args.list, "r") as fl:
            pheno = []
            for line in fl:
                pheno.append(line.rstrip())
        with open(disc_genes, "w") as fs:
            print("GENE\tCHR\tSNPnum\tsnpBP\tP\tlogP\tSNPeff\tallele\tdesignation\tstrand")
            for line in fr:
                col = line.split("\t")
                chrom = col[1]
                position = int(col[2])
                start = position - args.d
                end = position + args.d
                snpnum = col[0]
                pval = col[3]
                logP = col[4]
                snpeffect = col[5]
                alleleinfo = str(col[6]).replace("\n", "")
                
                #server = "http://rest.ensemblgenomes.org"
                server = "http://rest.ensembl.org"
                ext = "/overlap/region/oryza_sativa/{}:{}-{}:1?feature=gene".format(chrom, start, end)

                r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
                
                
                #print(r.json())
                if len(r.json()) != 0:
                    decoded=r.json()[0]
                    genes = decoded[u'gene_id']
                    description = decoded[u'description']
                    sense = decoded[u'description']

                genelist = (",").join(genes) #join all iterative elements by ,
                
                #use str.join() to convert multiple elments in a list into one string.
                keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
                link = "http://knetminer.rothamsted.ac.uk/riceknet/genome?"
                parameters = {"keyword":keyw1, "list":genelist}
                r = requests.get(link, params=parameters)
                
                #check if request is succesfsul.
                if not r.ok:
                    r.raise_for_status()
                    sys.exit()
                
                #printing out genetable to a file.
                decoded=r.json()[u'geneTable'].split("\t")
                #remove space or newline at the end.
                decoded=(decoded)[:-1]

                ncolumns=9
                #tabulate the list to be any dimension as long as it's dividable by 9 columns.
                genetable=np.array(decoded).reshape(len(decoded)//ncolumns, ncolumns)
                genetable = pd.DataFrame(genetable[1:,:], columns=genetable[0,:])

                genes=genetable[u'ACCESSION']
                knetscore=genetable[u'SCORE']
                genechr=genetable[u'CHRO']
                start=genetable[u'START']
                
                for i in genelist:
                    keyw2 = "+OR+".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
                    link="http://knetminer.rothamsted.ac.uk/riceknet/genepage?list={}&keyword={}".format(genes, keyw2)
                    r=requests.get(link)
                    print(r.url)
                    print("{}\t{}\t{}".format(genes, score, r.url))
                    
                print("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(gene, chrom, snpnum, position, pval, logP, snpeffect, alleleinfo, description, score, url))
                                   
    #end of get genes

# Learning decorators

In [1]:
def plus_one(number):
    return number + 1

add_one = plus_one
add_one(1)

2

#### creating a function inside a function

In [7]:
def addafterdouble(number):
    def double(number):
        return number * 2
    final= double(number) + 3
    return final

addafterdouble(5)

13

##### function as parameter to another function